In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
import tensorflow_io as tfio
import soundfile as sf
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from functools import partial

from data import fetch_metadata,load_wav_for_map, load_wav_16k_mono, bird_mapping_idx_to_name
from yamnet import Yamnet

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:


class ReduceMeanLayer(tf.keras.layers.Layer):
    
    def call(self, input):
        return tf.math.reduce_mean(input, axis=0)
model = tf.keras.models.load_model('./model_all_birds_v1.h5', custom_objects={'ReduceMeanLayer': ReduceMeanLayer})



OSError: SavedModel file does not exist at: ./model_all_birds_v1.h5/{saved_model.pbtxt|saved_model.pb}

In [5]:
birdsong_metadata = fetch_metadata()
dataset = birdsong_metadata.copy()

train_audio_paths, valid_audio_paths, train_labels, valid_labels = train_test_split(
    dataset[["filepath"]], dataset["label"], stratify=dataset["label"], test_size=0.2, random_state=101
)



In [15]:
errors = []

def load_wav_16k_mono_no_tf(filename):
    """ read in a waveform file and convert to 16 kHz mono """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
        file_contents,
        desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

for path, label in tqdm(list(zip(valid_audio_paths.values.squeeze(), valid_labels.values.squeeze()))):
    testing_wav_data = load_wav_16k_mono_no_tf(path)
    results = model(testing_wav_data)

    top_probs, top_idxs = tf.math.top_k(results, k=5)
    _, most_likely_idx = tf.math.top_k(results, k=1)
    top_labels = [bird_mapping_idx_to_name[idx] for idx in top_idxs.numpy()]
    if label not in top_labels:
        errors.append({
            'label': bird_mapping_idx_to_name.get(label),
            'filepath': path
        })

  1%|          | 37/3822 [00:04<16:27,  3.83it/s]

  1%|          | 38/3822 [00:05<19:09,  3.29it/s]

  1%|          | 39/3822 [00:05<19:41,  3.20it/s]

  1%|          | 40/3822 [00:06<21:03,  2.99it/s]

  1%|          | 41/3822 [00:06<23:52,  2.64it/s]

...

100%|█████████▉| 3818/3822 [26:34<00:01,  3.24it/s]

100%|█████████▉| 3819/3822 [26:34<00:00,  3.51it/s]

100%|█████████▉| 3820/3822 [26:34<00:00,  3.87it/s]

100%|█████████▉| 3821/3822 [26:34<00:00,  3.97it/s]

100%|██████████| 3822/3822 [26:35<00:00,  2.40it/s]


In [16]:
pd.DataFrame(errors)

,label,filepath
0,yellowhammer,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
1,great_tit,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
2,common_chaffinch,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
3,european_nightjar,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
4,eurasian_nuthatch,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
...,...,...
3817,tawny_owl,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
3818,common_chaffinch,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
3819,eurasian_collared_dove,/media/wwymak/Storage2/birdsong_dataset/xeno_c...
3820,great_tit,/media/wwymak/Storage2/birdsong_dataset/xeno_c...


In [8]:
len(val_ds)

TypeError: dataset length is unknown.